In [64]:
import common
import json

# Get a configured model
client = common.get_openai_client()

def validate_json(payload:str) -> bool:
    check = json.loads(payload)
    if "intent" in check:
        return True
    else:
        return False
    
def get_intent(payload:str) -> str:
    check = json.loads(payload)
    return check["intent"]

def mock_rag():
    return """Water is an inorganic compound with the chemical formula H2O. It is a transparent, tasteless, odorless,[c] and nearly colorless chemical substance, and it is the main constituent of Earth's hydrosphere and the fluids of all known living organisms (in which it acts as a solvent[20]). It is vital for all known forms of life, despite not providing food energy or organic micronutrients. Its chemical formula, H2O, indicates that each of its molecules contains one oxygen and two hydrogen atoms, connected by covalent bonds. The hydrogen atoms are attached to the oxygen atom at an angle of 104.45°.[21] "Water" is also the name of the liquid state of H2O at standard temperature and pressure."""

def process_for_intent(intentJson:str,input:str) -> str:
  if validate_json(intentJson):
      intent = get_intent(intentJson)      
      if intent == "AnswerQuestion":
          rag_template = input + "\n" + mock_rag() +"\n\nUse only the provided text."
          answer = common.Call_OpenAI(client,common.gpt_api_deployment,rag_template)
          print(f'A: {answer}')
      else:
          query_template = input
          answer = common.Call_OpenAI(client,common.gpt_api_deployment,query_template)
          print(f'A: {answer}')

In [67]:
intent_template = """An intent is an action from the list of defined intents.

Defined intents:
AnswerQuestion
QueryDatabase


What is the intent for the following request:
{{input}}

Output format:
{
  "intent": "intent",
}

Do not provide explanations. Output in JSON format only.
"""

input = "What is the chemical composition of water?"
template = common.render_template(intent_template,input=input)
intent = common.Call_OpenAI(client,common.gpt_api_deployment,template)

print(f'Q: {input}')
process_for_intent(intent,input)


input = "Write a SQL query to get the customer id and name with the highest balance. The customer table has the following fields: customer_id, customer_name, balance."
template = common.render_template(intent_template,input=input)
intent = common.Call_OpenAI(client,common.gpt_api_deployment,template)

print(f'\nQ: {input}')
process_for_intent(intent,input)



Q: What is the chemical composition of water?
A: Water has the chemical formula H2O and is an inorganic compound. It is transparent, tasteless, odorless, and nearly colorless. Water is the main constituent of Earth's hydrosphere and is found in the fluids of all known living organisms. Its chemical formula indicates that each molecule contains one oxygen and two hydrogen atoms connected by covalent bonds. The hydrogen atoms are attached to the oxygen atom at an angle of 104.45°. "Water" also refers to

Q: Write a SQL query to get the customer id and name with the highest balance. The customer table has the following fields: customer_id, customer_name, balance.
A: SELECT customer_id, customer_name 
FROM customer 
WHERE balance = (SELECT MAX(balance) FROM customer)
